In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
colors = sns.color_palette('viridis')

import numpy as np
np.set_printoptions(suppress=True)

from pyspark import SparkContext
from table import Table

# start spark with 1 worker thread
sc = SparkContext("local[*]")
sc.setLogLevel("ERROR")

21/11/27 18:13:48 WARN Utils: Your hostname, pasokon resolves to a loopback address: 127.0.1.1; using 192.168.1.96 instead (on interface wlo1)
21/11/27 18:13:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/27 18:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/27 18:13:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## study of scaling

hypothesis: the `event_type` field in *`machine_events`*
* *ADD (0)*: a machine became available to the cluster - all machines in the trace will have an ADD event.
* *REMOVE (1)*: a machine was removed from the cluster. Removals can occur due to failures or maintenance.
* *UPDATE (2)*: a machine available to the cluster had its available resources changed.

In [3]:
me = Table('machine_events', sc)
me.pprint(5)

,time,machine_id,event_type,platform_id,cpus,memory
0,0,5,0,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
1,0,6,0,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
2,0,7,0,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
3,0,10,0,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493
4,0,13,0,HofLGzk1Or/8Ildj2+Lqv0UGGvY82NLoni8+J/Yy0RU=,0.5,0.2493


In [4]:
#machine_events RDD
me_rdd = me.select(['machine_id','event_type']).map(lambda x: (x[0], int(x[1])))
me_rdd.take(5)

[('5', 0), ('6', 0), ('7', 0), ('10', 0), ('13', 0)]

In [34]:
import re
mzo = me_rdd.groupByKey().mapValues(list)\
      .mapValues(lambda x: ''.join(str(y) for y in x))\
      .filter(lambda x: (re.match(r'^(01)*[01]?$', x[1])))
mnzo = me_rdd.groupByKey().mapValues(list)\
      .mapValues(lambda x: ''.join(str(y) for y in x))\
      .filter(lambda x: re.match(r'^(01)*[01]?$', x[1]) is None)
      
assert me_rdd.groupByKey().count() == mzo.count() + mnzo.count()

In [8]:
te = Table('task_events', sc)
te.pprint(5)

,time,missing_info,job_id,task_index,machine_id,event_type,user,scheduling_class,priority,cpu_request,memory_request,disk_space_request,different_machines_restriction
0,0,2,3418309,0,4155527081,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,NA,NA,NA,NA
1,0,2,3418309,1,329150663,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,NA,NA,NA,NA
2,0,NA,3418314,0,3938719206,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,0.125,0.07446,0.0004244,0
3,0,NA,3418314,1,351618647,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,0.125,0.07446,0.0004244,0
4,0,2,3418319,0,431052910,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,NA,NA,NA,NA


In [12]:
te.rdd.count()

450146

In [36]:
te_rdd = te.select(
    ['machine_id','job_id', 'task_index','scheduling_class',
     'priority','cpu_request','memory_request','disk_space_request'])
te_mzo = mzo.join(te_rdd)
te_mnzo = mnzo.join(te_rdd)

In [44]:
te_mzo.count() + te_mnzo.count()

330575